In [1]:
from __future__ import division
import pandas as pd
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

### Creates design files for the reinforcement learning task

Design files are DataFrames with the following info per trial:
1. `stim_set`: Which stimulus set is presented? [0, 1, 2]
2. `correct_stim_lr`: What is the location (on the screen) of the winning stim? [0 = left, 1=right]
3. `p_win_left`: Probability of winning if left is chosen
4. `p_win_right'`: Probability of winning if right is chosen
5. `p_win_correct`: Probability of winning if correct answer is chosen
6. `p_win_incorrect`: Probability of winning if incorrect answer is chosen
7. `jitter_1`: Jitter duration for phase 1 (in s; see below)
8. `jitter_3`: Jitter duration for phase 3 (in s; see below)
9. `jitter_5`: Jitter duration for phase 5 (in s; see below)

Trial phases:
1. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
2. Cue (0.75s)
3. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
4. Stimulus (2s)
5. Stimulus choice (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
6. Feedback (0.5s)
7. ITI (jittered timing) `[0.25, 0.5, ..., 3.25s]`?

In [2]:
def get_settings(tr=2, verbose=True):
    p_win = [[.8, .2], [.7, .3], [.6, .4]]
    n_runs = 3
    n_sessions = 2
    if tr == 2:
        n_trials = 128
        jitters = [0.5, 0.75, 1, 1.25, 1.5]
        volumes_per_trial = 4
    
    n_trials_per_stimset = n_trials/len(p_win)
    trial_duration = volumes_per_trial*tr
    total_duration = trial_duration*n_trials
    total_duration_min = total_duration/60
    total_volumes = 1 + n_trials*volumes_per_trial
    
    if verbose:
        print('Settings:\n\n\
        Sessions: {n_sessions}\n\
        Runs per session: {n_runs}\n\
        Trials per run: {n_trials}\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {tr}*{trial_duration}*{n_trials} = {total_duration} seconds = {total_duration_min} min\n\
        Total number of volumes necessary: 1+{n_trials}*{volumes_per_trial} = {total_volumes} + warm-up pulses'.format(**locals()))
        
    return({'jitter': jitters,
            'n_trials': n_trials})

In [3]:
def generate_block_design(n_trials, jitters):
    
    stim_sets = [0, 1, 2]
    correct_stim_lr = [0, 1]
    cues = ['SPD', 'ACC']
    combs = list(itertools.product(stim_sets, correct_stim_lr, cues))
    
    # make basic df
    design = pd.DataFrame(combs * int(np.ceil((n_trials/len(combs)))), 
                          columns=['stimulus_set', 'correct_stim_lr', 'cue'])
    # randomize
    design = design.sample(frac=1).reset_index(drop=True)#.reset_index('trial_ID')
    n_trials_real = design.shape[0]
    design = design.iloc[:n_trials]
    
    if not n_trials == n_trials_real:
        print('WARNING: not totally balanced (%d not a multitude of %d)' %(n_trials, len(combs)))
    
    # Add probabilities (left/right and correct/incorrect - this is redundant, I know)
    design['p_win_left'] = None
    design['p_win_right'] = None
    design['p_win_correct'] = None
    design['p_win_incorrect'] = None
    for stim_set, p_win in zip([0, 1, 2], [.8, .7, .6]):
        p_lose = 1-p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_left'] = p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_right'] = p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_left'] = p_lose
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_right'] = p_lose
        design.loc[(design.stimulus_set==stim_set), 'p_win_correct'] = p_win
        design.loc[(design.stimulus_set==stim_set), 'p_win_incorrect'] = 1-p_win
    
    # Add jitters
    design['jitter_1'] = np.random.choice(jitters, size=n_trials, replace=True)
    design['jitter_3'] = np.random.choice(jitters, size=n_trials, replace=True)
    design['jitter_5'] = np.random.choice(jitters, size=n_trials, replace=True)
    
    return(design)

#generate_block_design(18, [0, 1, 2])

In [92]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.arange(1,n_subjects+1):
    designs_this_session = []

    for run in range(1,n_runs+1):
        settings = get_settings(tr=tr)
        design = generate_block_design(settings['n_trials'], settings['jitter'])
        design['block'] = run

        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    fn = 'sub-' + str(subject_id).zfill(2) + '_tr-' + str(tr) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
sub-01_tr-2_design


In [93]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,jitter_1,jitter_3,jitter_5,block
0,1,0,SPD,0.7,0.3,0.7,0.3,1.50,0.75,0.75,1
1,1,0,ACC,0.7,0.3,0.7,0.3,1.50,1.00,1.00,1
2,0,1,SPD,0.2,0.8,0.8,0.2,0.50,0.50,1.50,1
3,2,1,SPD,0.4,0.6,0.6,0.4,0.75,1.00,1.25,1
4,0,1,ACC,0.2,0.8,0.8,0.2,0.50,0.50,1.00,1
5,2,0,SPD,0.6,0.4,0.6,0.4,1.00,0.50,1.00,1
6,1,0,SPD,0.7,0.3,0.7,0.3,1.00,1.00,1.00,1
7,2,0,ACC,0.6,0.4,0.6,0.4,1.00,1.25,1.25,1
8,0,0,SPD,0.8,0.2,0.8,0.2,1.25,1.25,1.50,1
9,2,0,SPD,0.6,0.4,0.6,0.4,0.50,1.50,0.50,1


### design for debugging

In [4]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in ['DEBUG']: #np.arange(1,n_subjects+1):
    designs_this_session = []

    for run in range(1,n_runs+1):
        settings = get_settings(tr=tr)
        design = generate_block_design(settings['n_trials'], settings['jitter'])
        design['block'] = run

        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    fn = 'sub-' + str(subject_id).zfill(2) + '_tr-' + str(tr) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
sub-DEBUG_tr-2_design
